## ORIGINAL RECTANGULAR DOMAIN PROBLEMS WITH NEUMANN, ROBIN, DIRICHLET BOUNDARY CONDITIONS AND INITIAL CONDITION AS SINUSOIDAL WAVE.

In [ ]:
import numpy as np
import pandas as pd

# Constants
alpha_range = (0.005, 0.1)  # Range for alpha (thermal diffusivity)
Lx_range = (0.5, 2.0)  # Range for length in x-direction
Ly_range = (0.5, 2.0)  # Range for length in y-direction
n_modes = 5  # Number of modes for sinusoidal variations

# Function to generate initial condition with sinusoidal variations
def initial_condition(n=1, m=1):
    return f"T(x, y, 0) = sin({n}πx)sin({m}πy)", lambda x, y: np.sin(n * np.pi * x) * np.sin(m * np.pi * y)

# Boundary conditions
def dirichlet_conditions():
    return [
        "T(0, y, t) = 0",
        "T(Lx, y, t) = 0",
        "T(x, 0, t) = 0",
        "T(x, Ly, t) = 0"
    ]

def neumann_conditions():
    return [
        "∂T/∂x(0, y, t) = 0",
        "∂T/∂x(Lx, y, t) = 0",
        "∂T/∂y(x, 0, t) = 0",
        "∂T/∂y(x, Ly, t) = 0"
    ]

def robin_conditions(h=1, beta=0.1):
    return [
        f"{h} ∂T/∂x(0, y, t) + {beta} T(0, y, t) = 0",
        f"{h} ∂T/∂x(Lx, y, t) + {beta} T(Lx, y, t) = 0",
        f"{h} ∂T/∂y(x, 0, t) + {beta} T(x, 0, t) = 0",
        f"{h} ∂T/∂y(x, Ly, t) + {beta} T(x, Ly, t) = 0"
    ]

# Compute Fourier coefficients (b_nm)
def compute_fourier_coefficients(initial_condition_func, n, m, Lx, Ly, boundary_type):
    try:
        if boundary_type == "dirichlet":
            integrand = lambda x, y: initial_condition_func(x, y) * np.sin(n * np.pi * x / Lx) * np.sin(m * np.pi * y / Ly)
        elif boundary_type == "neumann":
            integrand = lambda x, y: initial_condition_func(x, y) * np.cos(n * np.pi * x / Lx) * np.cos(m * np.pi * y / Ly)
        elif boundary_type == "robin":
            integrand = lambda x, y: initial_condition_func(x, y) * np.sin(n * np.pi * x / Lx) * np.sin(m * np.pi * y / Ly)
        else:
            raise ValueError("Unsupported boundary condition type")
        
        # Double integration over the domain
        x = np.linspace(0, Lx, 100)
        y = np.linspace(0, Ly, 100)
        X, Y = np.meshgrid(x, y)
        integral = np.trapz(np.trapz(integrand(X, Y), x), y)
        b_nm = (4 / (Lx * Ly)) * integral  # Fourier coefficient
        return b_nm
    except Exception as e:
        print(f"Error computing Fourier coefficients for mode (n={n}, m={m}): {e}")
        return None

# Generate detailed solutions
def generate_detailed_solution(initial_condition_str, initial_condition_func, boundary_conditions_str, alpha, Lx, Ly, boundary_type):
    explanation = f"""
### Detailed Solution
"""
    # Add general solution template
    if boundary_type == "dirichlet":
        general_solution = "T(x, y, t) = Σ Σ [b_nm sin(nπx/Lx) sin(mπy/Ly) exp(-α (n²π²/Lx² + m²π²/Ly²) t)]"
    elif boundary_type == "neumann":
        general_solution = "T(x, y, t) = Σ Σ [b_nm cos(nπx/Lx) cos(mπy/Ly) exp(-α (n²π²/Lx² + m²π²/Ly²) t)]"
    elif boundary_type == "robin":
        general_solution = "T(x, y, t) = Σ Σ [b_nm sin(nπx/Lx) sin(mπy/Ly) exp(-α (n²π²/Lx² + m²π²/Ly²) t)]"
    else:
        return None  # Unsupported boundary type

    explanation += f"1. Start with the general solution:\n   {general_solution}\n\n"

    # Compute Fourier coefficients for first few modes
    explanation += "2. Compute the Fourier coefficients b_nm:\n"
    coefficients = []
    for n in range(1, n_modes + 1):
        for m in range(1, n_modes + 1):
            b_nm = compute_fourier_coefficients(initial_condition_func, n, m, Lx, Ly, boundary_type)
            if b_nm is not None:
                coefficients.append((n, m, b_nm))
                explanation += f"   b_({n},{m}) = {b_nm:.4f}\n"
            else:
                return None  # Skip problem if Fourier coefficients cannot be computed

    # Construct the solution using the first few terms
    explanation += "\n3. Substitute the coefficients into the solution:\n   T(x, y, t) ≈ "
    terms = [
        f"{b_nm:.4f} sin({n}πx/Lx) sin({m}πy/Ly) exp(-{alpha:.4f} * ({n}²π²/Lx² + {m}²π²/Ly²) t)"
        for n, m, b_nm in coefficients[:5]  # Use first 5 terms for simplicity
    ]
    explanation += " + ".join(terms) + "\n"
    return explanation

# Generate dataset for problems
questions = []
answers = []

def generate_problem():
    alpha = round(np.random.uniform(alpha_range[0], alpha_range[1]), 4)
    Lx = round(np.random.uniform(Lx_range[0], Lx_range[1]), 2)
    Ly = round(np.random.uniform(Ly_range[0], Ly_range[1]), 2)
    n = np.random.randint(1, n_modes + 1)
    m = np.random.randint(1, n_modes + 1)
    initial_condition_str, initial_condition_func = initial_condition(n, m)

    boundary_type = np.random.choice(["dirichlet", "neumann", "robin"])
    if boundary_type == "dirichlet":
        boundary_conditions_str = "\n".join(dirichlet_conditions())
    elif boundary_type == "neumann":
        boundary_conditions_str = "\n".join(neumann_conditions())
    elif boundary_type == "robin":
        boundary_conditions_str = "\n".join(robin_conditions())
    
    answer = generate_detailed_solution(initial_condition_str, initial_condition_func, boundary_conditions_str, alpha, Lx, Ly, boundary_type)
    if answer:
        question = f"Solve the heat equation on a rectangular domain with the following conditions:\nInitial Condition:\n{initial_condition_str}\nBoundary Conditions:\n{boundary_conditions_str}"
        return question, answer
    return None, None

# Generate 400 problems
for _ in range(300):
    question, answer = generate_problem()
    if question and answer:
        questions.append(question)
        answers.append(answer)

# Create pandas DataFrame
df = pd.DataFrame({"Question": questions, "Answer": answers})

# Save to CSV
df.to_csv("/Users/srimanthdhondy/Programs/Projects/IISCTask/heat_equation_rectangular_domain_all_domain_solutions.csv", index=False)

print("Dataset with 400 detailed problems generated and saved.")

## ORIGINAL UNIT SQUARE MESH WITH NEUMANN, ROBIN, DIRICHLET BOUNDARY CONDITIONS WITH PARABOLIC, GAUSSIAN, LINEAR COMBINATION OF SINE AND COSINE AS THE INTIAL CONDITIONS.

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Constants
alpha_range = (0.005, 0.1)
n_modes = 3  # Reduced n_modes
nx = 5  # Very coarse grid
ny = 5  # Very coarse grid
Lx = 1.0
Ly = 1.0
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)
nt = 50  # Reduced nt
dt = 0.01

# Function to generate various initial conditions (excluding sinusoidal)
def initial_condition(choice=None):
    choices = [2, 3, 4]
    if choice is None:
        choice = random.choice(choices)

    if choice == 2:  # Gaussian
        x0 = np.random.uniform(0.25, 0.75)
        y0 = np.random.uniform(0.25, 0.75)
        sigma_x = np.random.uniform(0.1, 0.2)
        sigma_y = np.random.uniform(0.1, 0.2)
        return f"T(x, y, 0) = exp(-((x-{x0:.2f})/{sigma_x:.2f})**2 - ((y-{y0:.2f})/{sigma_y:.2f})**2)", lambda x, y: np.exp(-((x - x0) / sigma_x)**2 - ((y - y0) / sigma_y)**2)
    elif choice == 3:  # Linear combination of cosines and sines
        n1 = np.random.randint(1, n_modes + 1)
        m1 = np.random.randint(1, n_modes + 1)
        n2 = np.random.randint(1, n_modes + 1)
        m2 = np.random.randint(1, n_modes + 1)
        a = np.random.uniform(0.5, 1.5)
        b = np.random.uniform(0.5, 1.5)
        return f"T(x, y, 0) = {a:.2f}*cos({n1}*pi*x)*cos({m1}*pi*y) + {b:.2f}*sin({n2}*pi*x)*sin({m2}*pi*y)", lambda x, y: a*np.cos(n1 * np.pi * x) * np.cos(m1 * np.pi * y) + b*np.sin(n2 * np.pi * x) * np.sin(m2 * np.pi * y)
    elif choice == 4: # Parabolic
        a = np.random.uniform(0.5, 1.5)
        b = np.random.uniform(0.5, 1.5)
        c = np.random.uniform(-0.5,0.5)
        return f"T(x, y, 0) = {a:.2f}*x*(1-x) + {b:.2f}*y*(1-y) + {c:.2f}", lambda x,y: a*x*(1-x) + b*y*(1-y) + c
    else:
        raise ValueError("Invalid initial condition choice")

# Boundary conditions (using numerical representation)
def apply_dirichlet(T):
    T[0, :] = 0
    T[-1, :] = 0
    T[:, 0] = 0
    T[:, -1] = 0
    return T

def apply_neumann(T):
    T[0, :] = T[1, :]
    T[-1, :] = T[-2, :]
    T[:, 0] = T[:, 1]
    T[:, -1] = T[:, -2]
    return T

def apply_robin(T, h=1, beta=0.1):
    T[0, :] = (h * T[1, :]) / (h + beta * dx)
    T[-1, :] = (h * T[-2, :]) / (h + beta * dx)
    T[:, 0] = (h * T[:, 1]) / (h + beta * dy)
    T[:, -1] = (h * T[:, -2]) / (h + beta * dy)
    return T

# Numerical solution using finite difference method (ADI)
def solve_heat_equation_numerical(initial_condition_func, alpha, boundary_type):
    T = np.zeros((nx, ny))
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y)
    T = initial_condition_func(X, Y)

    # Construct sparse matrices
    Ax = sparse.diags([1, -2, 1], [-1, 0, 1], shape=(nx-2, nx-2))
    Ax = (alpha * dt / dx**2) * Ax
    Ax = sparse.eye(nx-2) - Ax
    Axsparse = sparse.csc_matrix(Ax)

    By = sparse.diags([1, -2, 1], [-1, 0, 1], shape=(ny - 2, ny - 2))
    By = (alpha * dt / dy**2) * By
    By = sparse.eye(ny - 2) - By
    Bysparse = sparse.csc_matrix(By)

    for _ in range(nt):
        T_interior = T[1:-1, 1:-1].copy()

        # Apply x-direction implicit step
        for j in range(ny - 2):
            T[1:-1, j+1] = spsolve(Axsparse, T_interior[:,j])
        
        # Apply boundary conditions after x-step
        if boundary_type == "dirichlet":
            T = apply_dirichlet(T)
        elif boundary_type == "neumann":
            T = apply_neumann(T)
        elif boundary_type == "robin":
            T = apply_robin(T)
        
        T_interior = T[1:-1, 1:-1].copy()
        # Apply y-direction implicit step
        for i in range(nx - 2):
            T[i+1, 1:-1] = spsolve(Bysparse, T_interior[i,:].T)
        
        # Apply boundary conditions after y-step
        if boundary_type == "dirichlet":
            T = apply_dirichlet(T)
        elif boundary_type == "neumann":
            T = apply_neumann(T)
        elif boundary_type == "robin":
            T = apply_robin(T)

    return T

def generate_problem():
    alpha = round(np.random.uniform(alpha_range[0], alpha_range[1]), 4)
    n = np.random.randint(1, n_modes + 1)
    m = np.random.randint(1, n_modes + 1)
    initial_condition_str, initial_condition_func = initial_condition()

    boundary_type = np.random.choice(["dirichlet", "neumann", "robin"])
    if boundary_type == "dirichlet":
        boundary_conditions_str = "Dirichlet"
    elif boundary_type == "neumann":
        boundary_conditions_str = "Neumann"
    elif boundary_type == "robin":
        boundary_conditions_str = "Robin"

    numerical_solution = solve_heat_equation_numerical(initial_condition_func, alpha, boundary_type)

    question = f"Solve the heat equation on a unit square with:\nInitial Condition: {initial_condition_str}\nBoundary Conditions: {boundary_conditions_str}\nα = {alpha}"

    # Generate explanation
    explanation = f"We solve this using a finite difference method with an Alternating Direction Implicit (ADI) scheme.\n"
    explanation += f"1. Discretize the domain into a {nx}x{ny} grid with dx = {dx:.4f} and dy = {dy:.4f}.\n"
    explanation += f"2. The initial condition is discretized as T(i, j, 0) = {initial_condition_str}.\n"
    explanation += f"3. Apply the boundary conditions: {boundary_conditions_str}.\n"

    if boundary_type == "dirichlet":
        explanation += "   - T(0, j, t) = T(Lx, j, t) = T(i, 0, t) = T(i, Ly, t) = 0\n"
    elif boundary_type == "neumann":
        explanation += "   - ∂T/∂x(0, j, t) = ∂T/∂x(Lx, j, t) = ∂T/∂y(i, 0, t) = ∂T/∂y(i, Ly, t) = 0 (approximated numerically using central difference)\n"
    elif boundary_type == "robin":
        explanation += "   - h ∂T/∂x + βT = 0 at the boundaries (approximated numerically using forward/backward difference)\n"

    explanation += f"4. Time stepping is performed using ADI method with dt = {dt:.4f} for {nt} time steps.\n"
    explanation += "5. The resulting temperature distribution T(x, y, t) is approximated numerically.\n"
    explanation += f"Numerical Solution (represented as a {nx}x{ny} grid):\n"
    explanation += str(numerical_solution.tolist())

    return question, explanation

# Generate dataset
questions = []
answers = []
num_problems = 300
for _ in range(num_problems):
    question, answer = generate_problem()
    questions.append(question)
    answers.append(answer)

df = pd.DataFrame({"Question": questions, "Answer": answers})  # Changed column name
df.to_csv("/Users/srimanthdhondy/Programs/Projects/IISCTask/300_Unit_square_mesh_heat_equation_Various_Conditions.csv", index=False)

print(f"Dataset with {num_problems} numerical solutions and explanations generated and saved.")

## ORIGINAL CIRCULAR DOMAIN PROBLEMS WITH NEUMANN, ROBIN, DIRICHLET BOUNDARY CONDITIONS AND INITIAL CONDITION AS SINUSOIDAL WAVE.

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import jv, jvp, jn_zeros
from scipy.optimize import root_scalar

# Constants
alpha = 0.01  # Thermal diffusivity
r_max = 1.0   # Radius of the circular domain

# Radial grid points
r = np.linspace(0, r_max, 100)  # 100 radial points

# Initial condition (e.g., sine function in radial direction)
def sine_initial_condition_radial(a=1):
    return f"u(r, 0) = sin({a}r)", lambda r: np.sin(a * r)

def indicator_initial_condition_radial():
    return (
        "u(r, 0) = 1 if r ∈ [0.2, 0.8], else u(r, 0) = 0",
        lambda r: np.where((r >= 0.2) & (r <= 0.8), 1, 0),
    )

# Boundary conditions for the circular domain
def dirichlet_boundary_conditions():
    return ["u(1, θ, t) = 0"]

def neumann_boundary_conditions():
    return ["∂u/∂r |_(r=1) = 0"]

def robin_boundary_conditions(k=1.0, h=2.0):
    return [f"{k} ∂u/∂r |_(r=1) + {h} u(1, θ, t) = g(t)"]

# Compute eigenvalues (λ_n) and coefficients (b_nm)
def compute_eigenvalues_and_coefficients(boundary_type, initial_condition_func, n_terms=4, m_terms=4, k=1, h=2):
    coefficients = []
    for m in range(m_terms):  # Loop over angular modes
        if boundary_type == "dirichlet":
            lambdas = jn_zeros(m, n_terms)  # Zeros of J_m
        elif boundary_type == "neumann":
            lambdas = jvp(m, jn_zeros(m, n_terms), n=1)  # Zeros of J_m'
        elif boundary_type == "robin":
            lambdas = []
            for n in range(1, n_terms + 1):
                lambda_n = n * np.pi  # Approximate initial guess
                root = lambda x: k * jvp(m, x) + h * jv(m, x)
                success = False
                for scale in range(1, 10):  # Dynamically adjust bracket size
                    try:
                        result = root_scalar(root, bracket=[lambda_n - scale, lambda_n + scale], method='brentq')
                        lambdas.append(result.root)
                        success = True
                        break
                    except ValueError:
                        continue
                if not success:
                    print(f"Warning: Unable to find root for mode (m={m}, n={n}). Skipping.")
                    return None  # Indicate failure to compute eigenvalues
        else:
            raise ValueError("Unsupported boundary condition type")

        for n, lambda_n in enumerate(lambdas, start=1):
            # Compute the radial integral
            integrand = lambda r: initial_condition_func(r) * jv(m, lambda_n * r)
            radial_integral = np.trapz(integrand(r) * r, r)  # Multiply by 'r' for area element
            
            # Coefficient b_nm
            b_nm = 2 * radial_integral / (jv(m + 1, lambda_n)**2)
            coefficients.append((m, n, lambda_n, b_nm))
    return coefficients

# Generate solutions for different boundary conditions
def generate_solution_for_boundary_conditions(boundary_type, initial_condition_str, initial_condition_func, alpha, r_max, k=1, h=2):
    explanation = f"""    
### Detailed Solution
1. Start with the general solution for the heat equation on a circular domain:
   u(r, θ, t) = Σ Σ [b_nm J_m(λ_n r) exp(-α λ_n² t) cos(mθ)], where J_m is the Bessel function of the first kind.

2. Compute the eigenvalues λ_n based on the boundary condition:
"""
    # Compute eigenvalues and coefficients
    coefficients = compute_eigenvalues_and_coefficients(boundary_type, initial_condition_func, k=k, h=h)
    if coefficients is None:
        return None  # If eigenvalues cannot be computed, return None
    
    for m, n, lambda_n, b_nm in coefficients:
        explanation += f"   λ_({m},{n}) = {lambda_n:.4f}, b_({m},{n}) = {b_nm:.4f}\n"

    explanation += """
3. Substitute the eigenvalues and coefficients into the solution:
   u(r, θ, t) ≈ """
    terms = []
    for m, n, lambda_n, b_nm in coefficients[:5]:  # Limit to 5 terms for simplicity
        term = f"{b_nm:.4f} J_{m}({lambda_n:.4f} r) exp(-{alpha:.4f} * {lambda_n:.4f}² t) cos({m} θ)"
        terms.append(term)
    explanation += " + ".join(terms) + "\n"

    explanation += """
4. This provides the approximate solution for the given initial and boundary conditions.
"""
    return explanation

# Generate dataset
questions = []
answers = []

# Generate problems for different boundary conditions
boundary_types = ["dirichlet", "neumann", "robin"]
for boundary_type in boundary_types:
    for _ in range(150):  # Generate 100 problems for each boundary condition type
        new_alpha = round(np.random.uniform(0.005, 0.1), 4)  # Randomly vary alpha
        initial_condition_str, initial_condition_func = sine_initial_condition_radial(a=round(np.random.uniform(1, 5), 2))
        if boundary_type == "robin":
            k = round(np.random.uniform(0.5, 2.0), 2)
            h = round(np.random.uniform(0.5, 2.0), 2)
            answer = generate_solution_for_boundary_conditions(boundary_type, initial_condition_str, initial_condition_func, new_alpha, r_max, k, h)
        else:
            answer = generate_solution_for_boundary_conditions(boundary_type, initial_condition_str, initial_condition_func, new_alpha, r_max)
        
        if answer:  # Only add valid solutions
            question = f"Solve the heat equation on a circular domain with the following conditions:\nInitial Condition:\n{initial_condition_str}\nBoundary Conditions ({boundary_type.capitalize()})."
            questions.append(question)
            answers.append(answer)

# Create pandas DataFrame
df = pd.DataFrame({"Question": questions, "Answer": answers})

# Save to CSV
df.to_csv("/Users/srimanthdhondy/Programs/Projects/IISCTask/ORIGINAL_Modified_heat_equation_circular_domain_all_conditions_450.csv", index=False)

print("Dataset with solutions for Dirichlet, Neumann, and Robin boundary conditions saved to 'heat_equation_circular_domain_all_conditions_300.csv'.")

## ORIGINAL CIRCULAR DOMAIN PROBLEMS WITH NEUMANN, ROBIN, DIRICHLET BOUNDARY CONDITIONS WITH PARABOLIC, GAUSSIAN, LINEAR COMBINATION OF SINE AND COSINE AS THE INTIAL CONDITIONS.

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import scipy.special as special

# Constants
alpha_range = (0.005, 0.1)
n_modes = 3
nr = 5  # Number of radial points
ntheta = 5  # Number of angular points
r_max = 1.0  # Unit radius
dr = r_max / (nr - 1)
dtheta = 2 * np.pi / ntheta
nt = 50
dt = 0.01

# Function to generate various initial conditions (excluding sinusoidal, suitable for polar)
def initial_condition(choice=None):
    choices = [2, 3, 4]
    if choice is None:
        choice = random.choice(choices)

    if choice == 2:  # Gaussian
        r0 = np.random.uniform(0.25, 0.75)  # Center radius
        sigma_r = np.random.uniform(0.1, 0.2)
        return f"T(r, θ, 0) = exp(-((r-{r0:.2f})/{sigma_r:.2f})**2)", lambda r, theta: np.exp(-((r - r0) / sigma_r)**2)
    elif choice == 3:  # Bessel functions using scipy.special.jv (CORRECTED)
        m = np.random.randint(1, n_modes + 1)
        return f"T(r, θ, 0) = J{m}(r)", lambda r, theta: np.abs(special.jv(m, r))
    elif choice == 4:  # Parabolic
        a = np.random.uniform(0.5, 1.5)
        return f"T(r, θ, 0) = {a:.2f}*r*(1-r)", lambda r, theta: a*r*(1-r)
    else:
        raise ValueError("Invalid initial condition choice")

# Boundary conditions (using numerical representation)
def apply_dirichlet(T):
    T[0, :] = 0  # Center
    T[-1, :] = 0  # Outer edge
    return T

def apply_neumann(T):
    T[0, :] = T[1, :]
    T[-1, :] = T[-2, :]
    return T

def apply_robin(T, h=1, beta=0.1):
    T[0, :] = (h * T[1, :]) / (h + beta * dr)
    T[-1, :] = (h * T[-2, :]) / (h + beta * dr)
    return T

# Numerical solution using finite difference method (ADI in polar coordinates)
def solve_heat_equation_numerical(initial_condition_func, alpha, boundary_type):
    T = np.zeros((nr, ntheta))
    r = np.linspace(0, r_max, nr)
    theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
    R, Theta = np.meshgrid(r, theta)
    T = initial_condition_func(R, Theta)

    # Construct sparse matrices
    Ar = sparse.diags([1, -2, 1], [-1, 0, 1], shape=(nr - 2, nr - 2)).tolil()
    Ar = (alpha * dt / dr**2) * Ar
    Ar = sparse.eye(nr-2) - Ar
    Arsparse = sparse.csc_matrix(Ar)

    Atheta = sparse.diags([1, -2, 1], [-1, 0, 1], shape=(ntheta, ntheta)).tolil()
    Atheta[0, -1] = 1
    Atheta[-1, 0] = 1
    Atheta = (alpha * dt / (r_max**2 * dtheta**2)) * Atheta
    Atheta = Atheta.tocsc()


    for _ in range(nt):
        T_interior = T[1:-1, :].copy()

        # r-direction implicit step
        for j in range(ntheta):
            T[1:-1, j] = spsolve(Arsparse, T_interior[:, j])

        # Apply Boundary Conditions after r-step
        if boundary_type == "dirichlet":
            T = apply_dirichlet(T)
        elif boundary_type == "neumann":
            T = apply_neumann(T)
        elif boundary_type == "robin":
            T = apply_robin(T)

        T_interior = T[1:-1, :].copy()

        # theta-direction implicit step
        for i in range(nr - 2):
            T[i+1, :] = spsolve(Atheta, T_interior[i,:].T).T

        # Apply Boundary Conditions after theta-step
        if boundary_type == "dirichlet":
            T = apply_dirichlet(T)
        elif boundary_type == "neumann":
            T = apply_neumann(T)
        elif boundary_type == "robin":
            T = apply_robin(T)

    return T

def generate_problem():
    alpha = round(np.random.uniform(alpha_range[0], alpha_range[1]), 4)
    initial_condition_str, initial_condition_func = initial_condition()

    boundary_type = np.random.choice(["dirichlet", "neumann", "robin"])
    if boundary_type == "dirichlet":
        boundary_conditions_str = "Dirichlet"
    elif boundary_type == "neumann":
        boundary_conditions_str = "Neumann"
    elif boundary_type == "robin":
        boundary_conditions_str = "Robin"

    numerical_solution = solve_heat_equation_numerical(initial_condition_func, alpha, boundary_type)

    question = f"Solve the heat equation on a unit circle with:\nInitial Condition: {initial_condition_str}\nBoundary Conditions: {boundary_conditions_str}\nα = {alpha}"

    explanation = f"We solve this using a finite difference method with an Alternating Direction Implicit (ADI) scheme in polar coordinates.\n"
    explanation += f"1. Discretize the domain into a {nr}x{ntheta} grid with dr = {dr:.4f} and dθ = {dtheta:.4f}.\n"
    explanation += f"2. The initial condition is discretized as T(i, j, 0) = {initial_condition_str}.\n"
    explanation += f"3. Apply the boundary conditions: {boundary_conditions_str}.\n"

    if boundary_type == "dirichlet":
        explanation += "   - T(0, j, t) = T(r_max, j, t) = 0\n"
    elif boundary_type == "neumann":
        explanation += "   - ∂T/∂r(0, j, t) = ∂T/∂r(r_max, j, t) = 0 (approximated numerically using central difference)\n"
    elif boundary_type == "robin":
        explanation += "   - h ∂T/∂r + βT = 0 at the boundaries (approximated numerically using forward/backward difference)\n"

    explanation += f"4. Time stepping is performed using ADI method with dt = {dt:.4f} for {nt} time steps.\n"
    explanation += "5. The resulting temperature distribution T(r, θ, t) is approximated numerically.\n"
    explanation += f"Numerical Solution (represented as a {nr}x{ntheta} grid):\n"
    explanation += str(numerical_solution.tolist())

    return question, explanation

# Generate dataset
questions = []
answers = []
num_problems = 300
for _ in range(num_problems):
    question, answer = generate_problem()
    questions.append(question)
    answers.append(answer)

df = pd.DataFrame({"Question": questions, "Answer": answers})
df.to_csv("/Users/srimanthdhondy/Programs/Projects/IISCTask/300_Unit_circle_mesh_heat_equation_Various_Conditions.csv", index=False)

print(f"Dataset with {num_problems} numerical solutions and explanations generated and saved.")